In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.ensemble import RandomForestClassifier

In [9]:
train_df_orig = pd.read_excel('JDACS.xlsx',engine='openpyxl',sheet_name='Query result')
test_df_orig = pd.read_excel('JDACS.xlsx',engine='openpyxl',sheet_name='Missing Cancel Reason')

In [10]:
test_df_orig.drop('Cancel Reason',axis='columns',inplace=True)
test_df_1 = test_df_orig.copy()

In [4]:
train_df_orig.shape

(6398, 17)

In [5]:
test_df_orig.shape

(329, 16)

In [12]:
test_df_1['index'] = test_df_1.index.values
test_df_1['index'] = test_df_1['index'].apply(lambda x:'test'+str(x))
test_df_1.set_index(test_df_1['index'],inplace=True)
test_df_1.drop('index',axis=1,inplace=True)
test_df_1.head()

,Personal or Business,Business Number,Vehicle Type,Standard or Express,Platform,Cancelled by,Placement Day,Placement Time,Confirmation Day,Confirmation Time,Cancellation Day,Cancellation Time,Distance (KM),Pickup Lat,Pickup Long,scheduled
index,,,,,,,,,,,,,,,,
test0,Personal,NaN,Bike,Express,ios,Customer,1.0,15:23:44,1.0,16:40:47,1.0,16:43:50,4.0,-1.299370,36.822107,OnDemAND
test1,Business,20680.0,Bike,Express,desktop,Staff,1.0,14:21:46,1.0,14:22:20,1.0,14:30:48,17.0,-1.264490,36.789303,OnDemAND
test2,Business,17968.0,Bike,Express,desktop,Customer,1.0,15:13:20,NaN,NaN,1.0,15:56:34,18.0,-1.427056,36.961499,OnDemAND
test3,Personal,NaN,Bike,Standard,android,Customer,1.0,13:54:19,NaN,NaN,1.0,13:56:41,8.0,-1.301737,36.743936,OnDemAND
test4,Personal,NaN,Bike,Standard,android,Staff,1.0,13:24:05,1.0,15:57:20,1.0,17:14:46,13.0,-1.308098,36.920464,Scheduled


In [13]:
# merge train df and test df
merged_df = pd.concat([train_df_orig,test_df_1])

In [14]:
merged_df.head()

,Personal or Business,Business Number,Vehicle Type,Standard or Express,Platform,Cancel Reason,Cancelled by,Placement Day,Placement Time,Confirmation Day,Confirmation Time,Cancellation Day,Cancellation Time,Distance (KM),Pickup Lat,Pickup Long,scheduled
0,Personal,NaN,Bike,Express,android,No driver has been allocated to my request.,Customer,1.0,17:55:38,NaN,NaN,1.0,17:56:35,9.0,-1.313053,36.841437,OnDemAND
1,Personal,NaN,Bike,Express,android,My order is not ready.,Customer,1.0,12:26:27,1.0,12:27:36,1.0,12:28:44,14.0,-1.318959,36.853394,OnDemAND
2,Business,20680.0,Bike,Express,android,I placed the wrong locations.,Customer,1.0,13:21:18,1.0,13:21:34,1.0,13:47:29,16.0,-1.313854,36.706867,OnDemAND
3,Personal,NaN,Bike,Express,android,My order is not ready.,Customer,1.0,10:29:19,NaN,NaN,1.0,10:29:29,20.0,-1.206197,36.778776,OnDemAND
4,Personal,NaN,Bike,Standard,ios,No Partner,Staff,1.0,14:34:05,1.0,14:59:08,1.0,15:49:41,11.0,-1.296958,36.808166,OnDemAND


In [15]:
merged_df.tail()

,Personal or Business,Business Number,Vehicle Type,Standard or Express,Platform,Cancel Reason,Cancelled by,Placement Day,Placement Time,Confirmation Day,Confirmation Time,Cancellation Day,Cancellation Time,Distance (KM),Pickup Lat,Pickup Long,scheduled
test324,Personal,NaN,Bike,Express,android,NaN,Staff,7.0,00:00:04,NaN,NaN,7.0,03:00:48,0.0,-1.305507,36.775842,OnDemAND
test325,Personal,NaN,Bike,Standard,android,NaN,Staff,7.0,06:15:12,7.0,08:27:31,7.0,10:49:54,3.0,-1.285342,36.770167,Scheduled
test326,Personal,NaN,Bike,Standard,desktop,NaN,Customer,7.0,16:33:09,NaN,NaN,7.0,16:33:57,17.0,-1.274964,36.911953,OnDemAND
test327,Personal,NaN,Bike,Standard,android,NaN,Customer,7.0,13:00:13,NaN,NaN,7.0,13:20:01,10.0,-1.324390,36.849949,OnDemAND
test328,Personal,NaN,Bike,Standard,android,NaN,Customer,7.0,08:07:50,NaN,NaN,7.0,08:31:51,22.0,-1.297139,36.790044,OnDemAND


In [16]:
# drop columns with majority missing values
merged_df.drop(['Business Number','Confirmation Day','Confirmation Time'],axis='columns',inplace=True)

,Personal or Business,Vehicle Type,Standard or Express,Platform,Cancel Reason,Cancelled by,Placement Day,Placement Time,Cancellation Day,Cancellation Time,Distance (KM),Pickup Lat,Pickup Long,scheduled
0,Personal,Bike,Express,android,No driver has been allocated to my request.,Customer,1.0,17:55:38,1.0,17:56:35,9.0,-1.313053,36.841437,OnDemAND
1,Personal,Bike,Express,android,My order is not ready.,Customer,1.0,12:26:27,1.0,12:28:44,14.0,-1.318959,36.853394,OnDemAND
2,Business,Bike,Express,android,I placed the wrong locations.,Customer,1.0,13:21:18,1.0,13:47:29,16.0,-1.313854,36.706867,OnDemAND
3,Personal,Bike,Express,android,My order is not ready.,Customer,1.0,10:29:19,1.0,10:29:29,20.0,-1.206197,36.778776,OnDemAND
4,Personal,Bike,Standard,ios,No Partner,Staff,1.0,14:34:05,1.0,15:49:41,11.0,-1.296958,36.808166,OnDemAND
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
test324,Personal,Bike,Express,android,NaN,Staff,7.0,00:00:04,7.0,03:00:48,0.0,-1.305507,36.775842,OnDemAND
test325,Personal,Bike,Standard,android,NaN,Staff,7.0,06:15:12,7.0,10:49:54,3.0,-1.285342,36.770167,Scheduled
test326,Personal,Bike,Standard,desktop,NaN,Customer,7.0,16:33:09,7.0,16:33:57,17.0,-1.274964,36.911953,OnDemAND
test327,Personal,Bike,Standard,android,NaN,Customer,7.0,13:00:13,7.0,13:20:01,10.0,-1.324390,36.849949,OnDemAND


In [22]:
# same day delivery cancellations
merged_df_same = merged_df[merged_df['Cancellation Day'] == merged_df['Placement Day']].copy()

In [23]:
merged_df_same[['Placement Time','Cancellation Time']] = merged_df_same[['Placement Time','Cancellation Time']].apply(lambda x: pd.to_datetime(x,format='%H:%M:%S'))
merged_df_same['Wait Time'] = merged_df_same['Cancellation Time'] - merged_df_same['Placement Time']
merged_df_same['Wait Time'] = merged_df_same['Wait Time'].dt.total_seconds()

In [24]:
merged_df_same['Placement Hour'] = merged_df_same['Placement Time'].dt.hour
merged_df_same['Placement Minute'] = merged_df_same['Placement Time'].dt.minute
merged_df_same['Cancellation Hour'] = merged_df_same['Cancellation Time'].dt.hour
merged_df_same['Cancellation Minute'] = merged_df_same['Cancellation Time'].dt.minute

In [25]:
merged_df_same[['Placement Hour','Placement Minute','Cancellation Hour','Cancellation Minute']] = merged_df_same[['Placement Hour','Placement Minute','Cancellation Hour','Cancellation Minute']].apply(lambda x: x.astype('category'))

In [26]:
merged_df_same.head()

,Personal or Business,Vehicle Type,Standard or Express,Platform,Cancel Reason,Cancelled by,Placement Day,Placement Time,Cancellation Day,Cancellation Time,Distance (KM),Pickup Lat,Pickup Long,scheduled,Wait Time,Placement Hour,Placement Minute,Cancellation Hour,Cancellation Minute
0,Personal,Bike,Express,android,No driver has been allocated to my request.,Customer,1.0,1900-01-01 17:55:38,1.0,1900-01-01 17:56:35,9.0,-1.313053,36.841437,OnDemAND,57.0,17,55,17,56
1,Personal,Bike,Express,android,My order is not ready.,Customer,1.0,1900-01-01 12:26:27,1.0,1900-01-01 12:28:44,14.0,-1.318959,36.853394,OnDemAND,137.0,12,26,12,28
2,Business,Bike,Express,android,I placed the wrong locations.,Customer,1.0,1900-01-01 13:21:18,1.0,1900-01-01 13:47:29,16.0,-1.313854,36.706867,OnDemAND,1571.0,13,21,13,47
3,Personal,Bike,Express,android,My order is not ready.,Customer,1.0,1900-01-01 10:29:19,1.0,1900-01-01 10:29:29,20.0,-1.206197,36.778776,OnDemAND,10.0,10,29,10,29
4,Personal,Bike,Standard,ios,No Partner,Staff,1.0,1900-01-01 14:34:05,1.0,1900-01-01 15:49:41,11.0,-1.296958,36.808166,OnDemAND,4536.0,14,34,15,49


In [27]:
merged_df_same.dtypes

Personal or Business            object
Vehicle Type                    object
Standard or Express             object
Platform                        object
Cancel Reason                   object
Cancelled by                    object
Placement Day                  float64
Placement Time          datetime64[ns]
Cancellation Day               float64
Cancellation Time       datetime64[ns]
Distance (KM)                  float64
Pickup Lat                     float64
Pickup Long                    float64
scheduled                       object
Wait Time                      float64
Placement Hour                category
Placement Minute              category
Cancellation Hour             category
Cancellation Minute           category
dtype: object

In [30]:
merged_df_same.drop(['Placement Time','Cancellation Time'],axis='columns',inplace=True)

In [31]:
merged_df_same.head()

,Personal or Business,Vehicle Type,Standard or Express,Platform,Cancel Reason,Cancelled by,Placement Day,Cancellation Day,Distance (KM),Pickup Lat,Pickup Long,scheduled,Wait Time,Placement Hour,Placement Minute,Cancellation Hour,Cancellation Minute
0,Personal,Bike,Express,android,No driver has been allocated to my request.,Customer,1.0,1.0,9.0,-1.313053,36.841437,OnDemAND,57.0,17,55,17,56
1,Personal,Bike,Express,android,My order is not ready.,Customer,1.0,1.0,14.0,-1.318959,36.853394,OnDemAND,137.0,12,26,12,28
2,Business,Bike,Express,android,I placed the wrong locations.,Customer,1.0,1.0,16.0,-1.313854,36.706867,OnDemAND,1571.0,13,21,13,47
3,Personal,Bike,Express,android,My order is not ready.,Customer,1.0,1.0,20.0,-1.206197,36.778776,OnDemAND,10.0,10,29,10,29
4,Personal,Bike,Standard,ios,No Partner,Staff,1.0,1.0,11.0,-1.296958,36.808166,OnDemAND,4536.0,14,34,15,49


In [35]:
merged_df_same_encoded = pd.get_dummies(merged_df_same, columns=['Personal or Business','Vehicle Type','Standard or Express','Platform','Cancelled by','scheduled','Placement Hour','Placement Minute','Cancellation Hour','Cancellation Minute'])

In [36]:
merged_df_same_encoded

,Cancel Reason,Placement Day,Cancellation Day,Distance (KM),Pickup Lat,Pickup Long,Wait Time,Personal or Business_Business,Personal or Business_Personal,Vehicle Type_Bike,...,Cancellation Minute_50,Cancellation Minute_51,Cancellation Minute_52,Cancellation Minute_53,Cancellation Minute_54,Cancellation Minute_55,Cancellation Minute_56,Cancellation Minute_57,Cancellation Minute_58,Cancellation Minute_59
0,No driver has been allocated to my request.,1.0,1.0,9.0,-1.313053,36.841437,57.0,0,1,1,...,0,0,0,0,0,0,1,0,0,0
1,My order is not ready.,1.0,1.0,14.0,-1.318959,36.853394,137.0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
2,I placed the wrong locations.,1.0,1.0,16.0,-1.313854,36.706867,1571.0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
3,My order is not ready.,1.0,1.0,20.0,-1.206197,36.778776,10.0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
4,No Partner,1.0,1.0,11.0,-1.296958,36.808166,4536.0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
test324,NaN,7.0,7.0,0.0,-1.305507,36.775842,10844.0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
test325,NaN,7.0,7.0,3.0,-1.285342,36.770167,16482.0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
test326,NaN,7.0,7.0,17.0,-1.274964,36.911953,48.0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
test327,NaN,7.0,7.0,10.0,-1.324390,36.849949,1188.0,0,1,1,...,0,0,0,0,0,0,0,0,0,0


In [37]:
merged_df_same_encoded.dtypes

Cancel Reason              object
Placement Day             float64
Cancellation Day          float64
Distance (KM)             float64
Pickup Lat                float64
                           ...   
Cancellation Minute_55      uint8
Cancellation Minute_56      uint8
Cancellation Minute_57      uint8
Cancellation Minute_58      uint8
Cancellation Minute_59      uint8
Length: 189, dtype: object

In [46]:
test_index = merged_df_same_encoded.loc['test0':'test328'].index.values
test_df = merged_df_same_encoded.loc['test0':'test328'].copy()
train_df = merged_df_same_encoded.drop(test_index,axis='index')
train_df.head()

,Cancel Reason,Placement Day,Cancellation Day,Distance (KM),Pickup Lat,Pickup Long,Wait Time,Personal or Business_Business,Personal or Business_Personal,Vehicle Type_Bike,...,Cancellation Minute_50,Cancellation Minute_51,Cancellation Minute_52,Cancellation Minute_53,Cancellation Minute_54,Cancellation Minute_55,Cancellation Minute_56,Cancellation Minute_57,Cancellation Minute_58,Cancellation Minute_59
0,No driver has been allocated to my request.,1.0,1.0,9.0,-1.313053,36.841437,57.0,0,1,1,...,0,0,0,0,0,0,1,0,0,0
1,My order is not ready.,1.0,1.0,14.0,-1.318959,36.853394,137.0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
2,I placed the wrong locations.,1.0,1.0,16.0,-1.313854,36.706867,1571.0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
3,My order is not ready.,1.0,1.0,20.0,-1.206197,36.778776,10.0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
4,No Partner,1.0,1.0,11.0,-1.296958,36.808166,4536.0,0,1,1,...,0,0,0,0,0,0,0,0,0,0


In [47]:
test_df_encoded = test_df.drop(['Cancel Reason'],axis='columns')
test_df_encoded.head()

,Placement Day,Cancellation Day,Distance (KM),Pickup Lat,Pickup Long,Wait Time,Personal or Business_Business,Personal or Business_Personal,Vehicle Type_Bike,Vehicle Type_Standard Bike,...,Cancellation Minute_50,Cancellation Minute_51,Cancellation Minute_52,Cancellation Minute_53,Cancellation Minute_54,Cancellation Minute_55,Cancellation Minute_56,Cancellation Minute_57,Cancellation Minute_58,Cancellation Minute_59
test0,1.0,1.0,4.0,-1.299370,36.822107,4806.0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
test1,1.0,1.0,17.0,-1.264490,36.789303,542.0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
test2,1.0,1.0,18.0,-1.427056,36.961499,2594.0,1,0,1,0,...,0,0,0,0,0,0,1,0,0,0
test3,1.0,1.0,8.0,-1.301737,36.743936,142.0,0,1,1,0,...,0,0,0,0,0,0,1,0,0,0
test4,1.0,1.0,13.0,-1.308098,36.920464,13841.0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0


In [48]:
X_train = train_df.drop('Cancel Reason',axis='columns')
y_train = train_df['Cancel Reason']

In [49]:
X_train_split, X_test_split, y_train_split, y_test_split = train_test_split(X_train,y_train,test_size=0.2)

In [50]:
clf = RandomForestClassifier()
clf.fit(X_train_split, y_train_split)
clf.score(X_test_split, y_test_split)

0.5445378151260504

In [51]:
param_grid = {
    'n_estimators': [10,20,30],
    'max_depth': [6,10,20,30]
}

gridsearch = GridSearchCV(RandomForestClassifier(n_jobs=-1),
                         param_grid=param_grid,
                         scoring='accuracy', cv=3,
                         return_train_score=True, verbose=10)

gridsearch.fit(X_train_split, y_train_split)

Fitting 3 folds for each of 12 candidates, totalling 36 fits
[CV 1/3; 1/12] START max_depth=6, n_estimators=10...............................
[CV 1/3; 1/12] END max_depth=6, n_estimators=10;, score=(train=0.504, test=0.473) total time=  22.1s
[CV 2/3; 1/12] START max_depth=6, n_estimators=10...............................
[CV 2/3; 1/12] END max_depth=6, n_estimators=10;, score=(train=0.501, test=0.462) total time=   0.1s
[CV 3/3; 1/12] START max_depth=6, n_estimators=10...............................
[CV 3/3; 1/12] END max_depth=6, n_estimators=10;, score=(train=0.527, test=0.477) total time=   0.1s
[CV 1/3; 2/12] START max_depth=6, n_estimators=20...............................
[CV 1/3; 2/12] END max_depth=6, n_estimators=20;, score=(train=0.509, test=0.468) total time=   0.2s
[CV 2/3; 2/12] START max_depth=6, n_estimators=20...............................
[CV 2/3; 2/12] END max_depth=6, n_estimators=20;, score=(train=0.507, test=0.463) total time=   0.5s
[CV 3/3; 2/12] START max_dept

GridSearchCV(cv=3, estimator=RandomForestClassifier(n_jobs=-1),
             param_grid={'max_depth': [6, 10, 20, 30],
                         'n_estimators': [10, 20, 30]},
             return_train_score=True, scoring='accuracy', verbose=10)

In [52]:
pd.DataFrame(gridsearch.cv_results_).sort_values(by='rank_test_score')

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,mean_train_score,std_train_score
11,0.389697,0.017327,0.029448,0.007289,30,30,"{'max_depth': 30, 'n_estimators': 30}",0.528373,0.534048,0.515132,0.525851,0.007925,1,0.995586,0.995586,0.993380,0.994851,0.001040
7,0.331897,0.094833,0.020324,0.004262,20,20,"{'max_depth': 20, 'n_estimators': 20}",0.530895,0.520807,0.520807,0.524170,0.004756,2,0.924338,0.909836,0.908575,0.914250,0.007152
8,0.412310,0.034491,0.024426,0.006246,20,30,"{'max_depth': 20, 'n_estimators': 30}",0.519546,0.523960,0.518916,0.520807,0.002244,3,0.929382,0.928436,0.939155,0.932325,0.004845
10,0.295546,0.018801,0.017802,0.003150,30,20,"{'max_depth': 30, 'n_estimators': 20}",0.508197,0.510719,0.513241,0.510719,0.002059,4,0.992119,0.988020,0.983607,0.987915,0.003476
5,0.298520,0.035963,0.026731,0.000267,10,30,"{'max_depth': 10, 'n_estimators': 30}",0.511980,0.507566,0.510719,0.510088,0.001856,5,0.648487,0.637453,0.651009,0.645649,0.005887
4,0.264153,0.025498,0.023784,0.002773,10,20,"{'max_depth': 10, 'n_estimators': 20}",0.515132,0.493695,0.508197,0.505675,0.008932,6,0.638083,0.635561,0.639975,0.637873,0.001808
3,0.228125,0.032459,0.015547,0.006369,10,10,"{'max_depth': 10, 'n_estimators': 10}",0.501892,0.478562,0.508827,0.496427,0.012946,7,0.628625,0.604666,0.624212,0.619168,0.010411
6,0.254420,0.013845,0.021985,0.000817,20,10,"{'max_depth': 20, 'n_estimators': 10}",0.489912,0.486129,0.483607,0.486549,0.002591,8,0.901639,0.897541,0.877049,0.892077,0.010757
2,0.274445,0.022125,0.019097,0.004952,6,30,"{'max_depth': 6, 'n_estimators': 30}",0.485498,0.475410,0.494956,0.485288,0.007981,9,0.526797,0.524590,0.531526,0.527638,0.002893
9,0.232674,0.037563,0.020853,0.005034,30,10,"{'max_depth': 30, 'n_estimators': 10}",0.481715,0.495586,0.477932,0.485078,0.007590,10,0.970681,0.959332,0.970050,0.966688,0.005208


In [54]:
predictions = clf.predict(test_df_encoded)

In [60]:
pred_df = pd.DataFrame(predictions,index=test_df_encoded.index.values,columns=['Cancel Reason'])

In [64]:
(pred_df.shape, test_df_encoded.shape)

((275, 1), (275, 188))

In [75]:
same_day_pred_df = pd.merge(test_df_encoded,pred_df,left_index=True,right_index=True)


In [76]:
assert test_df.index.values.all() == pred_df.index.values.all()

In [77]:
# Cancellations that happened on a different day than the day of placement

In [78]:
merged_df_diff = merged_df[merged_df['Cancellation Day'] != merged_df['Placement Day']].copy()

In [79]:
merged_df_diff.shape

(504, 14)

In [80]:
merged_df_diff.tail()

,Personal or Business,Vehicle Type,Standard or Express,Platform,Cancel Reason,Cancelled by,Placement Day,Placement Time,Cancellation Day,Cancellation Time,Distance (KM),Pickup Lat,Pickup Long,scheduled
test306,Personal,Bike,Standard,android,NaN,Staff,6.0,22:04:54,7.0,03:00:48,19.0,-1.226308,36.739770,OnDemAND
test307,Business,Bike,Express,desktop,NaN,Staff,6.0,17:06:13,7.0,03:00:48,8.0,-1.295734,36.757895,OnDemAND
test308,Personal,Bike,Standard,android,NaN,Staff,6.0,22:03:56,7.0,03:00:48,19.0,-1.233180,36.736080,OnDemAND
test312,Personal,Bike,Express,ios,NaN,Staff,6.0,18:52:45,7.0,03:00:48,2.0,-1.256758,36.812851,OnDemAND
test323,Personal,Bike,Express,android,NaN,Staff,7.0,19:17:21,1.0,03:00:34,10.0,-1.320658,36.829238,OnDemAND


In [84]:
merged_df_diff['diff'] = merged_df_diff['Cancellation Day'] - merged_df_diff['Placement Day']
merged_df_diff['diff'].unique()

array([ 1.,  2.,  4.,  3.,  6.,  5., -1., -3., -4., -5., -6.])

In [88]:
merged_df_diff['diff']=merged_df_diff['diff'].map({1:1,2:2,3:3,4:4,5:5,-6:1,-5:2,-4:3,-3:4,-1:6})

In [90]:
merged_df_diff.head()

,Personal or Business,Vehicle Type,Standard or Express,Platform,Cancel Reason,Cancelled by,Placement Day,Placement Time,Cancellation Day,Cancellation Time,Distance (KM),Pickup Lat,Pickup Long,scheduled,diff
7,Personal,Bike,Standard,desktop,No driver has been allocated to my request.,Customer,1.0,16:10:52,2.0,07:44:42,21.0,-1.257130,36.699467,Scheduled,1.0
14,Business,Bike,Standard,desktop,No Partner,Staff,1.0,15:09:54,2.0,03:00:00,20.0,-1.256347,36.800263,OnDemAND,1.0
29,Business,Bike,Express,desktop,No Partner,Staff,1.0,10:32:37,2.0,03:00:00,6.0,-1.264490,36.784703,Scheduled,1.0
32,Personal,Bike,Express,android,Customer error,Staff,1.0,17:09:04,2.0,07:38:13,5.0,-1.290888,36.826661,OnDemAND,1.0
38,Personal,Bike,Express,desktop,I placed the wrong locations.,Customer,1.0,17:26:25,3.0,08:57:45,9.0,-1.324342,36.715765,Scheduled,2.0


In [91]:
merged_df_diff[['Placement Time','Cancellation Time']] = merged_df_diff[['Placement Time','Cancellation Time']].apply(lambda x: pd.to_datetime(x,format='%H:%M:%S'))
#merged_df_diff['Wait Time'] = merged_df_diff['Cancellation Time'] - merged_df_diff['Placement Time']
#merged_df_diff['Wait Time'] = merged_df_diff['Wait Time'].dt.total_seconds()

In [92]:
merged_df_diff['Placement Hour'] = merged_df_diff['Placement Time'].dt.hour
merged_df_diff['Placement Minute'] = merged_df_diff['Placement Time'].dt.minute
merged_df_diff['Cancellation Hour'] = merged_df_diff['Cancellation Time'].dt.hour
merged_df_diff['Cancellation Minute'] = merged_df_diff['Cancellation Time'].dt.minute

In [93]:
merged_df_diff[['Placement Hour','Placement Minute','Cancellation Hour','Cancellation Minute']] = merged_df_diff[['Placement Hour','Placement Minute','Cancellation Hour','Cancellation Minute']].apply(lambda x: x.astype('category'))

In [99]:
merged_df_diff.tail()

,Personal or Business,Vehicle Type,Standard or Express,Platform,Cancel Reason,Cancelled by,Placement Day,Cancellation Day,Distance (KM),Pickup Lat,Pickup Long,scheduled,diff,Placement Hour,Placement Minute,Cancellation Hour,Cancellation Minute
test306,Personal,Bike,Standard,android,NaN,Staff,6.0,7.0,19.0,-1.226308,36.739770,OnDemAND,1.0,22,4,3,0
test307,Business,Bike,Express,desktop,NaN,Staff,6.0,7.0,8.0,-1.295734,36.757895,OnDemAND,1.0,17,6,3,0
test308,Personal,Bike,Standard,android,NaN,Staff,6.0,7.0,19.0,-1.233180,36.736080,OnDemAND,1.0,22,3,3,0
test312,Personal,Bike,Express,ios,NaN,Staff,6.0,7.0,2.0,-1.256758,36.812851,OnDemAND,1.0,18,52,3,0
test323,Personal,Bike,Express,android,NaN,Staff,7.0,1.0,10.0,-1.320658,36.829238,OnDemAND,1.0,19,17,3,0


In [98]:
merged_df_diff.drop(['Placement Time','Cancellation Time'],axis='columns',inplace=True)

In [101]:
merged_df_diff_encoded = pd.get_dummies(merged_df_diff, columns=['Personal or Business','Vehicle Type','Standard or Express','Platform','Cancelled by','scheduled','Placement Hour','Placement Minute','Cancellation Hour','Cancellation Minute'])

In [114]:
merged_df_diff_encoded.tail()

,Cancel Reason,Placement Day,Cancellation Day,Distance (KM),Pickup Lat,Pickup Long,diff,Personal or Business_Business,Personal or Business_Personal,Vehicle Type_Bike,...,Cancellation Minute_49,Cancellation Minute_50,Cancellation Minute_51,Cancellation Minute_52,Cancellation Minute_53,Cancellation Minute_54,Cancellation Minute_55,Cancellation Minute_57,Cancellation Minute_58,Cancellation Minute_59
test306,NaN,6.0,7.0,19.0,-1.226308,36.739770,1.0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
test307,NaN,6.0,7.0,8.0,-1.295734,36.757895,1.0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
test308,NaN,6.0,7.0,19.0,-1.233180,36.736080,1.0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
test312,NaN,6.0,7.0,2.0,-1.256758,36.812851,1.0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
test323,NaN,7.0,1.0,10.0,-1.320658,36.829238,1.0,0,1,1,...,0,0,0,0,0,0,0,0,0,0


In [110]:
test_df_diff_encoded_idx = merged_df_diff_encoded.loc['test16':'test323'].index.values
test_df_diff_encoded = merged_df_diff_encoded.loc[test_df_diff_encoded_idx].copy()
train_df_diff_encoded = merged_df_diff_encoded.drop(test_df_diff_encoded_idx,axis='index')
train_df_diff_encoded.head()

,Cancel Reason,Placement Day,Cancellation Day,Distance (KM),Pickup Lat,Pickup Long,diff,Personal or Business_Business,Personal or Business_Personal,Vehicle Type_Bike,...,Cancellation Minute_49,Cancellation Minute_50,Cancellation Minute_51,Cancellation Minute_52,Cancellation Minute_53,Cancellation Minute_54,Cancellation Minute_55,Cancellation Minute_57,Cancellation Minute_58,Cancellation Minute_59
7,No driver has been allocated to my request.,1.0,2.0,21.0,-1.257130,36.699467,1.0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
14,No Partner,1.0,2.0,20.0,-1.256347,36.800263,1.0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
29,No Partner,1.0,2.0,6.0,-1.264490,36.784703,1.0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
32,Customer error,1.0,2.0,5.0,-1.290888,36.826661,1.0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
38,I placed the wrong locations.,1.0,3.0,9.0,-1.324342,36.715765,2.0,0,1,1,...,0,0,0,0,0,0,0,1,0,0


In [113]:
train_df_diff_encoded.shape,test_df_diff_encoded.shape

((450, 176), (54, 176))

In [116]:
test_df_diff_encoded.drop('Cancel Reason',axis='columns',inplace=True)

In [119]:
X_train_diff = train_df_diff_encoded.drop('Cancel Reason',axis='columns')
y_train_diff = train_df_diff_encoded['Cancel Reason']

In [120]:
X_train_split_diff, X_test_split_diff, y_train_split_diff, y_test_split_diff = train_test_split(X_train_diff,y_train_diff,test_size=0.2)

In [134]:
clf = RandomForestClassifier()
clf.fit(X_train_split_diff, y_train_split_diff)
clf.score(X_test_split_diff, y_test_split_diff)

0.8666666666666667

In [143]:
test_pred=pd.DataFrame(clf.predict(test_df_diff_encoded),index=test_df_diff_encoded.index.values,columns=['Cancel Reason'])

In [145]:
test_pred

,Cancel Reason
test16,No Partner
test18,No Partner
test32,No Partner
test59,No Partner
test60,No Partner
test66,No Partner
test93,No Partner
test94,No Partner
test96,No Partner
test97,No Partner


In [126]:
X_test_split_diff.isna().values.any()

False

In [127]:
nan_rows = X_train_split_diff[X_train_split_diff.isnull().T.any()]

In [128]:
nan_rows

,Placement Day,Cancellation Day,Distance (KM),Pickup Lat,Pickup Long,diff,Personal or Business_Business,Personal or Business_Personal,Vehicle Type_Bike,Vehicle Type_Standard Bike,...,Cancellation Minute_49,Cancellation Minute_50,Cancellation Minute_51,Cancellation Minute_52,Cancellation Minute_53,Cancellation Minute_54,Cancellation Minute_55,Cancellation Minute_57,Cancellation Minute_58,Cancellation Minute_59
485,1.0,7.0,6.0,-1.28913,36.813123,NaN,0,1,1,0,...,0,0,0,0,0,0,0,1,0,0


In [153]:
merged_df_diff.loc[test_df_diff_encoded_idx]['Cancel Reason'].fillna('No Partner')

In [156]:
merged_df_diff.loc[test_df_diff_encoded_idx]['Cancel Reason'].map({np.nan:'No Partner'})

test16     No Partner
test18     No Partner
test32     No Partner
test59     No Partner
test60     No Partner
test66     No Partner
test93     No Partner
test94     No Partner
test96     No Partner
test97     No Partner
test99     No Partner
test100    No Partner
test101    No Partner
test102    No Partner
test103    No Partner
test105    No Partner
test106    No Partner
test107    No Partner
test109    No Partner
test114    No Partner
test115    No Partner
test120    No Partner
test131    No Partner
test139    No Partner
test140    No Partner
test141    No Partner
test145    No Partner
test151    No Partner
test154    No Partner
test163    No Partner
test174    No Partner
test175    No Partner
test180    No Partner
test190    No Partner
test194    No Partner
test200    No Partner
test209    No Partner
test211    No Partner
test212    No Partner
test219    No Partner
test229    No Partner
test231    No Partner
test232    No Partner
test235    No Partner
test246    No Partner
test253   

In [158]:
diff_pred = pd.merge(test_df_diff_encoded,test_pred,left_index=True,right_index=True)

In [159]:
diff_pred

,Placement Day,Cancellation Day,Distance (KM),Pickup Lat,Pickup Long,diff,Personal or Business_Business,Personal or Business_Personal,Vehicle Type_Bike,Vehicle Type_Standard Bike,...,Cancellation Minute_50,Cancellation Minute_51,Cancellation Minute_52,Cancellation Minute_53,Cancellation Minute_54,Cancellation Minute_55,Cancellation Minute_57,Cancellation Minute_58,Cancellation Minute_59,Cancel Reason
test16,1.0,2.0,9.0,-1.272185,36.806102,1.0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,No Partner
test18,1.0,2.0,44.0,-1.102428,37.009867,1.0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,No Partner
test32,1.0,2.0,6.0,-1.259126,36.788049,1.0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,No Partner
test59,1.0,2.0,23.0,-1.376470,36.767165,1.0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,No Partner
test60,1.0,2.0,0.0,-1.327090,36.872394,1.0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,No Partner
test66,1.0,2.0,17.0,-1.269799,36.850803,1.0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,No Partner
test93,1.0,2.0,0.0,-1.328790,36.869994,1.0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,No Partner
test94,1.0,2.0,0.0,-1.329090,36.875294,1.0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,No Partner
test96,1.0,2.0,0.0,-1.326190,36.870894,1.0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,No Partner
test97,1.0,2.0,0.0,-1.329890,36.873994,1.0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,No Partner


In [160]:
same_day_pred_df

,Placement Day,Cancellation Day,Distance (KM),Pickup Lat,Pickup Long,Wait Time,Personal or Business_Business,Personal or Business_Personal,Vehicle Type_Bike,Vehicle Type_Standard Bike,...,Cancellation Minute_51,Cancellation Minute_52,Cancellation Minute_53,Cancellation Minute_54,Cancellation Minute_55,Cancellation Minute_56,Cancellation Minute_57,Cancellation Minute_58,Cancellation Minute_59,Cancel Reason
test0,1.0,1.0,4.0,-1.299370,36.822107,4806.0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,No driver has been allocated to my request.
test1,1.0,1.0,17.0,-1.264490,36.789303,542.0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,No Partner
test2,1.0,1.0,18.0,-1.427056,36.961499,2594.0,1,0,1,0,...,0,0,0,0,0,1,0,0,0,No driver has been allocated to my request.
test3,1.0,1.0,8.0,-1.301737,36.743936,142.0,0,1,1,0,...,0,0,0,0,0,1,0,0,0,No driver has been allocated to my request.
test4,1.0,1.0,13.0,-1.308098,36.920464,13841.0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,No Partner
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
test324,7.0,7.0,0.0,-1.305507,36.775842,10844.0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,No Partner
test325,7.0,7.0,3.0,-1.285342,36.770167,16482.0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,No Partner
test326,7.0,7.0,17.0,-1.274964,36.911953,48.0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,No driver has been allocated to my request.
test327,7.0,7.0,10.0,-1.324390,36.849949,1188.0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,No driver has been allocated to my request.


In [166]:
final_pred=pd.concat([same_day_pred_df,diff_pred])
final_pred['idx'] = final_pred.index.values
final_pred['idx'] = final_pred['idx'].apply(lambda x:x.replace('test',''))
final_pred.set_index('idx',inplace=True)

In [169]:
test_cancel_reason=final_pred['Cancel Reason']

In [175]:
final_pred['Cancel Reason']

idx
0      No driver has been allocated to my request.
1                                       No Partner
2      No driver has been allocated to my request.
3      No driver has been allocated to my request.
4                                       No Partner
                          ...                     
306                                     No Partner
307                                     No Partner
308                                     No Partner
312                                     No Partner
323                                     No Partner
Name: Cancel Reason, Length: 329, dtype: object

In [168]:
final_pred.shape

(329, 190)

In [171]:
test_df_1['Cancel Reason'] = test_cancel_reason

In [173]:
test_df_1.drop('Cancel Reason',inplace=True,axis='columns')

In [174]:
test_df_1

,Personal or Business,Business Number,Vehicle Type,Standard or Express,Platform,Cancelled by,Placement Day,Placement Time,Confirmation Day,Confirmation Time,Cancellation Day,Cancellation Time,Distance (KM),Pickup Lat,Pickup Long,scheduled
index,,,,,,,,,,,,,,,,
test0,Personal,NaN,Bike,Express,ios,Customer,1.0,15:23:44,1.0,16:40:47,1.0,16:43:50,4.0,-1.299370,36.822107,OnDemAND
test1,Business,20680.0,Bike,Express,desktop,Staff,1.0,14:21:46,1.0,14:22:20,1.0,14:30:48,17.0,-1.264490,36.789303,OnDemAND
test2,Business,17968.0,Bike,Express,desktop,Customer,1.0,15:13:20,NaN,NaN,1.0,15:56:34,18.0,-1.427056,36.961499,OnDemAND
test3,Personal,NaN,Bike,Standard,android,Customer,1.0,13:54:19,NaN,NaN,1.0,13:56:41,8.0,-1.301737,36.743936,OnDemAND
test4,Personal,NaN,Bike,Standard,android,Staff,1.0,13:24:05,1.0,15:57:20,1.0,17:14:46,13.0,-1.308098,36.920464,Scheduled
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
test324,Personal,NaN,Bike,Express,android,Staff,7.0,00:00:04,NaN,NaN,7.0,03:00:48,0.0,-1.305507,36.775842,OnDemAND
test325,Personal,NaN,Bike,Standard,android,Staff,7.0,06:15:12,7.0,08:27:31,7.0,10:49:54,3.0,-1.285342,36.770167,Scheduled
test326,Personal,NaN,Bike,Standard,desktop,Customer,7.0,16:33:09,NaN,NaN,7.0,16:33:57,17.0,-1.274964,36.911953,OnDemAND


In [178]:
test_df_orig.index.values

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
       130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
       143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
       156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168,
       169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 18

In [179]:
final_pred.index.values

array(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12',
       '13', '14', '15', '17', '19', '20', '21', '22', '23', '24', '25',
       '26', '27', '28', '29', '30', '31', '33', '34', '35', '36', '37',
       '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48',
       '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '61',
       '62', '63', '64', '65', '67', '68', '69', '70', '71', '72', '73',
       '74', '75', '76', '77', '78', '79', '80', '81', '82', '83', '84',
       '85', '86', '87', '88', '89', '90', '91', '92', '95', '98', '104',
       '108', '110', '111', '112', '113', '116', '117', '118', '119',
       '121', '122', '123', '124', '125', '126', '127', '128', '129',
       '130', '132', '133', '134', '135', '136', '137', '138', '142',
       '143', '144', '146', '147', '148', '149', '150', '152', '153',
       '155', '156', '157', '158', '159', '160', '161', '162', '164',
       '165', '166', '167', '168', '169', '170', '171', '172', 

In [180]:
test_df_orig.shape

(329, 16)

In [192]:
final_pred['Cancel Reason'].values

array(['No driver has been allocated to my request.', 'No Partner',
       'No driver has been allocated to my request.',
       'No driver has been allocated to my request.', 'No Partner',
       'The driver is too far away.',
       'No driver has been allocated to my request.',
       'I placed the wrong locations.', 'My order is not ready.',
       'The driver is too far away.', 'I placed the wrong locations.',
       'No Partner', 'I placed the wrong locations.',
       'My order is not ready.',
       'No driver has been allocated to my request.',
       'No driver has been allocated to my request.',
       'My order is not ready.', 'My order is not ready.',
       'No driver has been allocated to my request.',
       'No driver has been allocated to my request.',
       'My order is not ready.',
       'No driver has been allocated to my request.', 'No Partner',
       'No Partner', 'No driver has been allocated to my request.',
       'My order is not ready.', 'No Partner',
   

In [193]:
test_df_orig['Cancel Reason'] = final_pred['Cancel Reason'].values

In [191]:
test_df_orig.drop('Cancel Reason',axis=1,inplace=True)

In [194]:
test_df_orig

,Personal or Business,Business Number,Vehicle Type,Standard or Express,Platform,Cancelled by,Placement Day,Placement Time,Confirmation Day,Confirmation Time,Cancellation Day,Cancellation Time,Distance (KM),Pickup Lat,Pickup Long,scheduled,Cancel Reason
0,Personal,NaN,Bike,Express,ios,Customer,1.0,15:23:44,1.0,16:40:47,1.0,16:43:50,4.0,-1.299370,36.822107,OnDemAND,No driver has been allocated to my request.
1,Business,20680.0,Bike,Express,desktop,Staff,1.0,14:21:46,1.0,14:22:20,1.0,14:30:48,17.0,-1.264490,36.789303,OnDemAND,No Partner
2,Business,17968.0,Bike,Express,desktop,Customer,1.0,15:13:20,NaN,NaN,1.0,15:56:34,18.0,-1.427056,36.961499,OnDemAND,No driver has been allocated to my request.
3,Personal,NaN,Bike,Standard,android,Customer,1.0,13:54:19,NaN,NaN,1.0,13:56:41,8.0,-1.301737,36.743936,OnDemAND,No driver has been allocated to my request.
4,Personal,NaN,Bike,Standard,android,Staff,1.0,13:24:05,1.0,15:57:20,1.0,17:14:46,13.0,-1.308098,36.920464,Scheduled,No Partner
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
324,Personal,NaN,Bike,Express,android,Staff,7.0,00:00:04,NaN,NaN,7.0,03:00:48,0.0,-1.305507,36.775842,OnDemAND,No Partner
325,Personal,NaN,Bike,Standard,android,Staff,7.0,06:15:12,7.0,08:27:31,7.0,10:49:54,3.0,-1.285342,36.770167,Scheduled,No Partner
326,Personal,NaN,Bike,Standard,desktop,Customer,7.0,16:33:09,NaN,NaN,7.0,16:33:57,17.0,-1.274964,36.911953,OnDemAND,No Partner
327,Personal,NaN,Bike,Standard,android,Customer,7.0,13:00:13,NaN,NaN,7.0,13:20:01,10.0,-1.324390,36.849949,OnDemAND,No Partner


In [195]:
test_df_orig.to_excel('jdacs-filled.xlsx')